<a href="https://colab.research.google.com/github/iankim20/NGS-Research/blob/main/merging_variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
%cd "/content/drive/My Drive/vcf data"

/content/drive/My Drive/vcf data


In [ ]:
pip install scikit-allel

     |████████████████████████████████| 10.8MB 4.1MB/s 
  Created wheel for scikit-allel: filename=scikit_allel-1.3.2-cp36-cp36m-linux_x86_64.whl size=4708227 sha256=6e5f1f03dff62c915b8177c07db21f94e0ef84fbef239dc0b6d68cf1a439bd3b
  Stored in directory: /root/.cache/pip/wheels/f7/25/d0/32ddaaccfe4b1c20e701fad83533a13ff0c4b9809baa99d4bf
Successfully built scikit-allel


In [ ]:
import pandas as pd
import numpy as np
import os
 
# import scikit-allel
import allel
# check which version is installed
print(allel.__version__)

1.3.2


In [ ]:
chr_list = ['chr'+str(x) for x in range(1,23)] + ['chrX', 'chrY']
vcf_files=[]
vcf_files = [f for f in os.listdir('./scRNA_91nodes/') if f.endswith('.vcf')]


In [ ]:
file0 = vcf_files[1]

'SRR5957353_final.vcf'

In [ ]:
rna0 = allel.read_vcf(file0)

columns = ['CHROM', 'POS','ID','REF','ALT','QUAL','DP', 'variant']
rna0_df = pd.DataFrame(columns=columns)

In [ ]:
genotypes = rna0['calldata/GT']

merge_gt = []
for i in range(len(genotypes)):
  gt = genotypes[i]
  if np.array_equal(gt[0],[-1,-1]) and np.array_equal(gt[1],[-1,-1]):
    merge_gt.append([-1,-1])
  elif not(np.array_equal(gt[0],[-1,-1])) and np.array_equal(gt[1],[-1,-1]):
    merge_gt.append(gt[0])
  elif np.array_equal(gt[0],[-1,-1]) and not(np.array_equal(gt[1],[-1,-1])):
    merge_gt.append(gt[1]) 
  elif np.array_equal(gt[0],[-1,-1]) and np.array_equal(gt[1],[-1,-1]):
    merge_gt.append([NaN, NaN])

merge_gt = np.asarray(merge_gt)

In [ ]:
rna0_df['CHROM'] = rna0['variants/CHROM']
# rna0_df['FILTER'] = rna0['variants/FILTER_PASS']
rna0_df['POS'] = rna0['variants/POS']
rna0_df['ID'] = rna0['variants/ID']
rna0_df['REF'] = rna0['variants/REF']
rna0_df['ALT'] = rna0['variants/ALT'][:,0]
rna0_df['QUAL'] = rna0['variants/QUAL']
rna0_df['DP'] = allel.read_vcf(file0, fields=['DP'])['variants/DP']
rna0_df['variant'] = merge_gt


,CHROM,POS,ID,REF,ALT,QUAL,DP,variant
0,chr1,19608399,.,C,A,65.279999,6,"[-1, -1]"
1,chr1,19608436,.,T,C,63.279999,6,"[-1, -1]"
2,chr1,27649605,.,G,A,152.029999,4,"[-1, -1]"
3,chr1,27649611,.,C,T,152.029999,4,"[-1, -1]"
4,chr1,36090653,.,G,A,21.799999,7,"[0, 1]"
...,...,...,...,...,...,...,...,...
269,chr11,47252068,.,G,A,18.590000,2,"[-1, -1]"
270,chr11,59774294,.,T,C,32.740002,2,"[-1, -1]"
271,chr11,62676637,.,G,GTCCCAGC,98.250000,7,"[-1, -1]"
272,chr11,62676643,.,A,C,107.279999,6,"[-1, -1]"
